In [1]:
#!/usr/bin/python3.7

In [2]:
import time
import sklearn
import numpy as np
import pandas as pd

import sys
sys.path.append("..")
from baggingPU import BaggingClassifierPU

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random

In [3]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

# import data

In [4]:
#df_raw = pd.read_csv('../data/w-dependence.csv')

df_raw = pd.read_csv('../data/1place-independence.csv')

#df_raw = pd.read_csv('../data/w-related.csv')


df_raw['label'] = df_raw['label'].astype("int")
print(df_raw.label.value_counts())
print('Has null values', df_raw.isnull().values.any())

0    10000
1    10000
Name: label, dtype: int64
Has null values False


In [5]:
df_raw.head(10)

,p1,p2,p3,p4,label
0,1,0,0,0,1
1,0,1,1,0,1
2,1,0,1,0,1
3,0,0,0,1,1
4,0,1,2,0,1
5,1,0,2,0,1
6,0,0,1,1,1
7,0,1,3,0,1
8,1,0,3,0,1
9,0,0,2,1,1


In [6]:
print(df_raw.iloc[:10000,:])
df_max = np.amax(df_raw.iloc[:10000,:])
print(df_max)

      p1  p2    p3  p4  label
0      1   0     0   0      1
1      0   1     1   0      1
2      1   0     1   0      1
3      0   0     0   1      1
4      0   1     2   0      1
...   ..  ..   ...  ..    ...
9995   1   0  3332   0      1
9996   0   0  3331   1      1
9997   0   1  3333   0      1
9998   1   0  3333   0      1
9999   0   0  3332   1      1

[10000 rows x 5 columns]
p1          1
p2          1
p3       3333
p4          1
label       1
dtype: int64


In [7]:
print(df_raw.iloc[10000:,:])
print(np.amax(df_raw.iloc[10000:,[2]]))

         p1    p2    p3    p4  label
10000     0     0     0     0      0
10001     0     0     1     0      0
10002     0     1     0     0      0
10003     0     1     0     1      0
10004     0     1     1     1      0
...     ...   ...   ...   ...    ...
19995  1948  3267  3248    22      0
19996   678  2207  1707  1330      0
19997   796  3271  1813  1477      0
19998  2163   614  2392  2879      0
19999  1420  2443  2561  1150      0

[10000 rows x 5 columns]
p3    3333
dtype: int64


In [8]:
neg_process = df_raw.iloc[10000:,:].copy()
print(neg_process)

for i in range(10000):
    if neg_process.iloc[i,2] > df_max[0]:
        neg_process.iloc[i,2]  = int(neg_process.iloc[i,2] / df_max[0])

         p1    p2    p3    p4  label
10000     0     0     0     0      0
10001     0     0     1     0      0
10002     0     1     0     0      0
10003     0     1     0     1      0
10004     0     1     1     1      0
...     ...   ...   ...   ...    ...
19995  1948  3267  3248    22      0
19996   678  2207  1707  1330      0
19997   796  3271  1813  1477      0
19998  2163   614  2392  2879      0
19999  1420  2443  2561  1150      0

[10000 rows x 5 columns]


In [9]:
neg_process.drop_duplicates(inplace=True)
print(neg_process)
print(len(neg_process))
print(np.amax(neg_process))

         p1    p2    p3    p4  label
10000     0     0     0     0      0
10001     0     0     1     0      0
10002     0     1     0     0      0
10003     0     1     0     1      0
10004     0     1     1     1      0
...     ...   ...   ...   ...    ...
19995  1948  3267  3248    22      0
19996   678  2207  1707  1330      0
19997   796  3271  1813  1477      0
19998  2163   614  2392  2879      0
19999  1420  2443  2561  1150      0

[10000 rows x 5 columns]
10000
p1       3333
p2       3332
p3       3333
p4       3333
label       0
dtype: int64


In [10]:
df_process = pd.concat([df_raw.iloc[:10000,:], neg_process])
print(df_process)

         p1    p2    p3    p4  label
0         1     0     0     0      1
1         0     1     1     0      1
2         1     0     1     0      1
3         0     0     0     1      1
4         0     1     2     0      1
...     ...   ...   ...   ...    ...
19995  1948  3267  3248    22      0
19996   678  2207  1707  1330      0
19997   796  3271  1813  1477      0
19998  2163   614  2392  2879      0
19999  1420  2443  2561  1150      0

[20000 rows x 5 columns]


In [11]:
x_data = df_process.iloc[:,:-1]
y_data = df_process.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [12]:
x_train_input = pd.concat([x_train.iloc[:,:], y_train],axis=1)
x_test_input = pd.concat([x_test.iloc[:,:],y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [13]:
x_train_input.head(10)

,p1,p2,p3,p4,label
8118,0,0,2705,1,1
10171,0,0,2141,0,0
9399,0,0,3132,1,1
12669,1,1,2336,0,0
18809,1457,3115,942,1111,0
13869,0,0,1609,0,0
3320,1,0,1107,0,1
14689,1,1,1319,0,0
13087,1,1,2692,1,0
15992,515,833,2507,1884,0


In [14]:
x_test_input.head(10)

,p1,p2,p3,p4,label
14356,1,1,3189,1,0
3439,0,1,1147,0,1
12153,1,0,2606,1,0
15029,2966,1336,754,1574,0
18549,3088,1378,1096,3034,0
15762,622,795,773,1953,0
12313,0,1,1788,1,0
16034,704,1577,717,2627,0
11496,1,0,1418,1,0
5653,0,1,1885,0,1


In [15]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 5000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

print(X)
print(y)

         p1    p2    p3    p4
8118      0     0  2705     1
10171     0     0  2141     0
9399      0     0  3132     1
12669     1     1  2336     0
18809  1457  3115   942  1111
...     ...   ...   ...   ...
13927     1     1  1066     0
919       0     1   307     0
5699      1     0  1900     0
10742     0     1  2930     1
16921  2113   604  1171  1381

[16000 rows x 4 columns]
8118     0
10171    0
9399     0
12669    0
18809    0
        ..
13927    0
919      0
5699     0
10742    0
16921    0
Name: label, Length: 16000, dtype: int64


In [16]:
pd.Series(y).value_counts()

0    12923
1     3077
Name: label, dtype: int64

In [17]:
print('- %d samples and %d features' % (X.shape))
print('- %d positive out of %d total before hiding labels' % (sum(df_process.label), len(df_process.label)))
print('- %d positive out of %d total after hiding labels' % (sum(y), len(y)))

- 16000 samples and 4 features
- 10000 positive out of 20000 total before hiding labels
- 3077 positive out of 16000 total after hiding labels


# Trainning directly

In [18]:
print('Training XGboost model ...')

import xgboost as xgb

model = xgb.XGBClassifier()
model.fit(X, y)

print('Done')

Training XGboost model ...
[02:30:33] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Done


In [19]:
print('---- {} ----'.format('XGboost model'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- XGboost model ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive       7333.0        744.0 
None

Precision:  1.0
Recall:  0.0921134084437291
Accuracy:  0.5416875
f1_score:  0.16868835732910098


# Training by bagging

In [21]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()

from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier()

from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(30,30), random_state=1,max_iter=300)

from sklearn import svm
model4 = svm.LinearSVC()

from sklearn.naive_bayes import GaussianNB
model5 = GaussianNB()

from sklearn.ensemble import RandomForestClassifier
model6 = RandomForestClassifier(n_estimators = 50,n_jobs = -1)

import xgboost as xgb
model7 = xgb.XGBClassifier()


In [20]:
import logging
from FCM import FCM

model8 = FCM(n_clusters=2)
model8.set_logger(tostdout=True, level=logging.DEBUG)

In [22]:
model_list = [model1, model2, model3, model4, model5, model6, model7, model8]

In [22]:
print('Training Logistics Regression model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model1, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)

    pre = model.predict(X)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    
    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, pre), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, pre))
    print('Recall: ', recall_score(y_orig, pre))
    print('Accuracy: ', accuracy_score(y_orig, pre))
    print('f1_score: ', f1_score(y_orig, pre))
   
    #test data
    pre_test = model.predict(x_test)
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test,pre_test ), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, pre_test))
    print('Recall: ', recall_score(y_test, pre_test))
    print('Accuracy: ', accuracy_score(y_test, pre_test))
    print('f1_score: ', f1_score(y_test, pre_test))
    
    
    precision_train += precision_score(y_orig, pre)
    recall_train += recall_score(y_orig, pre)
    accuracy_train += accuracy_score(y_orig, pre)
    f1_train += f1_score(y_orig, pre)
    
    precision_test += precision_score(y_test,pre_test)
    recall_test += recall_score(y_test, pre_test)
    accuracy_test += accuracy_score(y_test, pre_test)
    f1_test += f1_score(y_test, pre_test)
    
print("===========================")
print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Logistics Regression model...
Training bagging classifier...
Done!
Time: 1.3797878599980322
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7148.0        775.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9124491640307275
Recall:  1.0
Accuracy:  0.9515625
f1_score:  0.9542205682556559
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       1868.0        209.0 
           true_positive          0.0       1923.0 
None

Precision:  0.9019699812382739
Recall:  1.0
Accuracy:  0.94775
f1_score:  0.9484586929716399
Training bagging classifier...
Done!
Time: 0.3440840660005051
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7148.0        775.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9124491640307275
Recall:  1.0


In [23]:
print('Training Decision Tree model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model2,
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pre = model.predict(X)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    
    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, pre), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, pre))
    print('Recall: ', recall_score(y_orig, pre))
    print('Accuracy: ', accuracy_score(y_orig, pre))
    print('f1_score: ', f1_score(y_orig, pre))
   
    #test data
    pre_test = model.predict(x_test)
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test,pre_test ), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, pre_test))
    print('Recall: ', recall_score(y_test, pre_test))
    print('Accuracy: ', accuracy_score(y_test, pre_test))
    print('f1_score: ', f1_score(y_test, pre_test))
    
    
    precision_train += precision_score(y_orig, pre)
    recall_train += recall_score(y_orig, pre)
    accuracy_train += accuracy_score(y_orig, pre)
    f1_train += f1_score(y_orig, pre)
    
    precision_test += precision_score(y_test,pre_test)
    recall_test += recall_score(y_test, pre_test)
    accuracy_test += accuracy_score(y_test, pre_test)
    f1_test += f1_score(y_test, pre_test)
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Decision Tree model...
Training bagging classifier...
Done!
Time: 0.271345574001316
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7903.0         20.0 
           true_positive       1761.0       6316.0 
None

Precision:  0.9968434343434344
Recall:  0.781973505014238
Accuracy:  0.8886875
f1_score:  0.8764309997918546
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2071.0          6.0 
           true_positive        441.0       1482.0 
None

Precision:  0.9959677419354839
Recall:  0.7706708268330733
Accuracy:  0.88825
f1_score:  0.8689533861037819
Training bagging classifier...
Done!
Time: 0.25757665199853363
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7916.0          7.0 
           true_positive       1741.0       6336.0 
None

Precision:  0.99889642

In [24]:
print('Training MLP model directly ...')

Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model3,
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pre = model.predict(X)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    
    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, pre), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, pre))
    print('Recall: ', recall_score(y_orig, pre))
    print('Accuracy: ', accuracy_score(y_orig, pre))
    print('f1_score: ', f1_score(y_orig, pre))
   
    #test data
    pre_test = model.predict(x_test)
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test,pre_test ), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, pre_test))
    print('Recall: ', recall_score(y_test, pre_test))
    print('Accuracy: ', accuracy_score(y_test, pre_test))
    print('f1_score: ', f1_score(y_test, pre_test))
    
    
    precision_train += precision_score(y_orig, pre)
    recall_train += recall_score(y_orig, pre)
    accuracy_train += accuracy_score(y_orig, pre)
    f1_train += f1_score(y_orig, pre)
    
    precision_test += precision_score(y_test,pre_test)
    recall_test += recall_score(y_test, pre_test)
    accuracy_test += accuracy_score(y_test, pre_test)
    f1_test += f1_score(y_test, pre_test)
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training MLP model directly ...
Training bagging classifier...
Done!
Time: 4.689668144997995
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       6704.0       1219.0 
           true_positive         76.0       8001.0 
None

Precision:  0.8677874186550976
Recall:  0.9905905658041352
Accuracy:  0.9190625
f1_score:  0.9251315256980979
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       1744.0        333.0 
           true_positive         20.0       1903.0 
None

Precision:  0.8510733452593918
Recall:  0.9895995839833593
Accuracy:  0.91175
f1_score:  0.9151238278432315
Training bagging classifier...
Done!
Time: 1.7995265620011196
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       3997.0       3926.0 
           true_positive          0.0       8077.0 
None

Precision:  0.67291510

In [25]:
print('Training SVM model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model4,
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pre = model.predict(X)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    
    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, pre), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, pre))
    print('Recall: ', recall_score(y_orig, pre))
    print('Accuracy: ', accuracy_score(y_orig, pre))
    print('f1_score: ', f1_score(y_orig, pre))
   
    #test data
    pre_test = model.predict(x_test)
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test,pre_test ), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, pre_test))
    print('Recall: ', recall_score(y_test, pre_test))
    print('Accuracy: ', accuracy_score(y_test, pre_test))
    print('f1_score: ', f1_score(y_test, pre_test))
    
    
    precision_train += precision_score(y_orig, pre)
    recall_train += recall_score(y_orig, pre)
    accuracy_train += accuracy_score(y_orig, pre)
    f1_train += f1_score(y_orig, pre)
    
    precision_test += precision_score(y_test,pre_test)
    recall_test += recall_score(y_test, pre_test)
    accuracy_test += accuracy_score(y_test, pre_test)
    f1_test += f1_score(y_test, pre_test)
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training SVM model...
Training bagging classifier...
Done!
Time: 0.5488632550004695
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7509.0        414.0 
           true_positive       6980.0       1097.0 
None

Precision:  0.7260092653871608
Recall:  0.13581775411662747
Accuracy:  0.537875
f1_score:  0.2288277012932833
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       1956.0        121.0 
           true_positive       1663.0        260.0 
None

Precision:  0.6824146981627297
Recall:  0.13520540821632865
Accuracy:  0.554
f1_score:  0.22569444444444445
Training bagging classifier...
Done!
Time: 0.5476903010021488
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7441.0        482.0 
           true_positive       6492.0       1585.0 
None

Precision:  0.7668118045476536


In [26]:
print('Training Naive Bayesianayes model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model5,
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pre = model.predict(X)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    
    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, pre), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, pre))
    print('Recall: ', recall_score(y_orig, pre))
    print('Accuracy: ', accuracy_score(y_orig, pre))
    print('f1_score: ', f1_score(y_orig, pre))
   
    #test data
    pre_test = model.predict(x_test)
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test,pre_test ), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, pre_test))
    print('Recall: ', recall_score(y_test, pre_test))
    print('Accuracy: ', accuracy_score(y_test, pre_test))
    print('f1_score: ', f1_score(y_test, pre_test))
    
    
    precision_train += precision_score(y_orig, pre)
    recall_train += recall_score(y_orig, pre)
    accuracy_train += accuracy_score(y_orig, pre)
    f1_train += f1_score(y_orig, pre)
    
    precision_test += precision_score(y_test,pre_test)
    recall_test += recall_score(y_test, pre_test)
    accuracy_test += accuracy_score(y_test, pre_test)
    f1_test += f1_score(y_test, pre_test)
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)
    

Training Naive Bayesianayes model...
Training bagging classifier...
Done!
Time: 0.3822890859992185
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       4024.0       3899.0 
           true_positive         27.0       8050.0 
None

Precision:  0.6736965436438196
Recall:  0.9966571746935744
Accuracy:  0.754625
f1_score:  0.8039548586837113
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       1012.0       1065.0 
           true_positive          5.0       1918.0 
None

Precision:  0.6429768689239022
Recall:  0.9973998959958398
Accuracy:  0.7325
f1_score:  0.7818997146351405
Training bagging classifier...
Done!
Time: 0.2680901570020069
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       4024.0       3899.0 
           true_positive         27.0       8050.0 
None

Precision:  0.6736

In [27]:
print('Training Random Forest model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model6,
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pre = model.predict(X)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    
    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, pre), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, pre))
    print('Recall: ', recall_score(y_orig, pre))
    print('Accuracy: ', accuracy_score(y_orig, pre))
    print('f1_score: ', f1_score(y_orig, pre))
   
    #test data
    pre_test = model.predict(x_test)
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test,pre_test ), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, pre_test))
    print('Recall: ', recall_score(y_test, pre_test))
    print('Accuracy: ', accuracy_score(y_test, pre_test))
    print('f1_score: ', f1_score(y_test, pre_test))
    
    
    precision_train += precision_score(y_orig, pre)
    recall_train += recall_score(y_orig, pre)
    accuracy_train += accuracy_score(y_orig, pre)
    f1_train += f1_score(y_orig, pre)
    
    precision_test += precision_score(y_test,pre_test)
    recall_test += recall_score(y_test, pre_test)
    accuracy_test += accuracy_score(y_test, pre_test)
    f1_test += f1_score(y_test, pre_test)
    

print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10) 

Training Random Forest model...
Training bagging classifier...
Done!
Time: 1.9298605490002956
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7922.0          1.0 
           true_positive       1050.0       7027.0 
None

Precision:  0.9998577120091064
Recall:  0.8700012380834469
Accuracy:  0.9343125
f1_score:  0.9304203905991394
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive        288.0       1635.0 
None

Precision:  1.0
Recall:  0.8502340093603744
Accuracy:  0.928
f1_score:  0.9190556492411467
Training bagging classifier...
Done!
Time: 1.5711006799974712
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7922.0          1.0 
           true_positive       1080.0       6997.0 
None

Precision:  0.9998571020291512
Recall:

In [28]:
print('Training XGboost model directly ...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model7,
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)
    pre = model.predict(X)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    
    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, pre), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, pre))
    print('Recall: ', recall_score(y_orig, pre))
    print('Accuracy: ', accuracy_score(y_orig, pre))
    print('f1_score: ', f1_score(y_orig, pre))
   
    #test data
    pre_test = model.predict(x_test)
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test,pre_test ), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, pre_test))
    print('Recall: ', recall_score(y_test, pre_test))
    print('Accuracy: ', accuracy_score(y_test, pre_test))
    print('f1_score: ', f1_score(y_test, pre_test))
    
    
    precision_train += precision_score(y_orig, pre)
    recall_train += recall_score(y_orig, pre)
    accuracy_train += accuracy_score(y_orig, pre)
    f1_train += f1_score(y_orig, pre)
    
    precision_test += precision_score(y_test,pre_test)
    recall_test += recall_score(y_test, pre_test)
    accuracy_test += accuracy_score(y_test, pre_test)
    f1_test += f1_score(y_test, pre_test)
    
print("===========================")
print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)   

Training XGboost model directly ...
Training bagging classifier...
Done!
Time: 4.787300621999748
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7922.0          1.0 
           true_positive        109.0       7968.0 
None

Precision:  0.9998745137407454
Recall:  0.986504890429615
Accuracy:  0.993125
f1_score:  0.993144708961735
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       2077.0          0.0 
           true_positive         29.0       1894.0 
None

Precision:  1.0
Recall:  0.984919396775871
Accuracy:  0.99275
f1_score:  0.9924024102698454
Training bagging classifier...
Done!
Time: 4.781931743000314
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       7920.0          3.0 
           true_positive        110.0       7967.0 
None

Precision:  0.9996235884567126
Recall:

In [23]:
X = np.array(X.values.tolist())
y = np.array(y.values.tolist())
y_orig = np.array(y_orig.values.tolist())
x_test = np.array(x_test.values.tolist())
y_test = np.array(y_test.values.tolist())


print('Training FCM model...')
Time = 0.0
precision_train = 0.0
recall_train = 0.0
accuracy_train = 0.0
f1_train = 0.0

precision_test = 0.0
recall_test = 0.0
accuracy_test = 0.0
f1_test = 0.0

for i in range(10):

    print('Training bagging classifier...')
    pu_start = time.perf_counter()
    model = BaggingClassifierPU(model8, 
                             n_jobs = -1, 
                             max_samples = sum(y) 
                        )
    model.fit(X, y)

    pre = model.predict(X)
    pu_end = time.perf_counter()
    print('Done!')
    print('Time:', pu_end - pu_start)
    Time += (pu_end - pu_start)
    
    #train data
    print('---- {} ----'.format('PU Bagging Train data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_orig, pre), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_orig, pre))
    print('Recall: ', recall_score(y_orig, pre))
    print('Accuracy: ', accuracy_score(y_orig, pre))
    print('f1_score: ', f1_score(y_orig, pre))
   
    #test data
    pre_test = model.predict(x_test)
    print('---- {} ----'.format('PU Bagging Test data'))
    print(print_cm(sklearn.metrics.confusion_matrix(y_test,pre_test ), labels=['negative', 'positive']))
    print('')
    print('Precision: ', precision_score(y_test, pre_test))
    print('Recall: ', recall_score(y_test, pre_test))
    print('Accuracy: ', accuracy_score(y_test, pre_test))
    print('f1_score: ', f1_score(y_test, pre_test))
    
    
    precision_train += precision_score(y_orig, pre)
    recall_train += recall_score(y_orig, pre)
    accuracy_train += accuracy_score(y_orig, pre)
    f1_train += f1_score(y_orig, pre)
    
    precision_test += precision_score(y_test,pre_test)
    recall_test += recall_score(y_test, pre_test)
    accuracy_test += accuracy_score(y_test, pre_test)
    f1_test += f1_score(y_test, pre_test)
    
print("===========================")
print("The final results:")
print("Time:" , Time / 10)
print("precision_train:" , precision_train / 10)
print("recall_train:" , recall_train / 10)
print("accuracy_train:" , accuracy_train / 10)
print("f1_train:" , f1_train / 10)
print("precision_test:" , precision_test / 10)
print("recall_test:" , recall_test / 10)
print("accuracy_test:" , accuracy_test / 10)
print("f1_test:" , f1_test / 10)

Training FCM model...
Training bagging classifier...
Done!
Time: 62.37270295061171
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       3984.0       3939.0 
           true_positive          0.0       8077.0 
None

Precision:  0.6721870838881492
Recall:  1.0
Accuracy:  0.7538125
f1_score:  0.8039615786592346
---- PU Bagging Test data ----
                        pred_negative        pred_positive 
           true_negative       1002.0       1075.0 
           true_positive          0.0       1923.0 
None

Precision:  0.6414276184122748
Recall:  1.0
Accuracy:  0.73125
f1_score:  0.781548465758992
Training bagging classifier...
Done!
Time: 63.54615320824087
---- PU Bagging Train data ----
                        pred_negative        pred_positive 
           true_negative       3984.0       3939.0 
           true_positive          0.0       8077.0 
None

Precision:  0.6721870838881492
Recall:  1.0
Accuracy:  0.7538125